In [38]:
import gc
import os
import itertools
import pickle
import re
import time
from random import choice, choices
from functools import reduce
from tqdm import tqdm
from itertools import cycle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from functools import reduce
from itertools import cycle
from scipy import stats
from scipy.stats import skew, kurtosis
from sklearn import metrics, model_selection, preprocessing, linear_model, ensemble, decomposition, tree
import lightgbm as lgb
import optuna
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


import sys
sys.path.append('../../')
from utils import *

## Prepro: change names to numbers

In [39]:
train_feats = pd.read_csv('../../feature_eng/output/train_double_corr_786feats.csv')
train_score = pd.read_csv('../../data/train_scores.csv')

if 'score' not in train_feats.columns:
    train_feats = pd.merge(train_feats, train_score, on='id', how='left')

train_feats = train_feats[~train_feats['id'].isin(['3e10785d'])]
train_feats.replace([np.inf, -np.inf], np.nan, inplace=True)
cols = train_feats.columns

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train_feats['score'])

score_transformed = le.transform(train_feats['score'])

## Keep columns

In [40]:
# Options:
# "count_bursts", "word_sent_parag_agg", "pressed_keys", "paussed_features"
# "segments_visit", "paragraph_ratios", "time_gaps", "cursor_word_changes"
# "punctuation", "key_mouse", "time_feat", "fft", "avg_event_per_minute", 
# "avg_char_deletion_per_minute", "avg_char_insert_per_minute"
# "IKI_word", "IKI_sentence", "IKI_paragraph", "latencies", "IWD"
# "idle"

endings_to_keep = ["count_bursts", "word_sent_parag_agg", "idle", "latencies", "avg_char_deletion_per_minute", "avg_char_insert_per_minute"]
endings_to_keep += ["id"]

dir_to_save = "low_lgb1"
stratification = True
n_jobs = 4
noise = True
data_augmentation = False

iterations = 5
n_splits = 3


try:
    os.mkdir(dir_to_save)
except:
    print('Already exists')
    sys.exit()

cols_to_keep = [col for col in train_feats.columns if col.endswith(tuple(endings_to_keep)) or col in ['id', 'score']]

with open(f'{dir_to_save}/feature_list.pkl', 'wb') as f:
    pickle.dump(cols_to_keep, f)

train_feats = train_feats[cols_to_keep]
print(len(train_feats.columns)-2)

178


In [41]:
counter = 0
new_columns = {}
for col in cols:
    if col != 'id' and col != 'score':
        new_columns[col] = counter
        counter += 1

train_feats.rename(columns=new_columns, inplace=True)

In [42]:
target_col = ['score']
drop_cols = ['id']
train_cols = [col for col in train_feats.columns if col not in target_col + drop_cols]

In [43]:
# def data_augmentation(x, y, pct=0.75, scale=0.05):
#     # Determine the number of rows to create
#     num_rows = x.shape[0]
#     num_new_rows = int(num_rows * pct)

#     # Randomly select rows from the original DataFrame
#     indices = np.random.choice(x.index, size=num_new_rows)
#     new_rows_x = x.loc[indices]

#     # Add Gaussian noise to each numerical column
#     for col in new_rows_x.select_dtypes(include=[np.number]).columns:
#         noise = np.random.normal(loc=0.0, scale=scale, size=num_new_rows)
#         new_rows_x[col] += new_rows_x[col] * noise

#     # Append the new rows to the original DataFrame
#     x = pd.concat([x, new_rows_x], ignore_index=True)

#     # Do the same for y
#     new_rows_y = y.loc[indices]
#     y = pd.concat([y, new_rows_y], ignore_index=True)

#     return x, y

In [44]:
# def objective(trial,train_x,train_y,val_x,val_y,RANDOM_STATE=41, data_augmentation=False):

#     params = {
#         'reg_lambda': trial.suggest_float('reg_lambda', 1e-4, 10.0, log=True),
#         'reg_alpha': trial.suggest_float('reg_alpha', 1e-4, 10.0, log=True),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1), #trial.suggest_categorical('colsample_bytree', [0.4,0.7,1.0]),
#         'colsample_bynode': trial.suggest_float('colsample_bynode', 0.5, 1), #trial.suggest_categorical('colsample_bynode', [0.4,0.7,1.0]),
#         'subsample': trial.suggest_float('subsample', 0.5, 1),# trial.suggest_categorical('subsample', [0.4,0.7,1.0]),
#         'learning_rate': trial.suggest_float('learning_rate', 1e-4, 5e-2, log=True), #trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
#         'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17,20]),
#         'min_child_samples': trial.suggest_categorical('min_child_samples', [7,9,11,13,15,17,20]),
#         'num_leaves': trial.suggest_int('num_leaves', 10, 38, step=4),
#         'n_estimators': trial.suggest_int('n_estimators', 10, 100, step=10),
#     }

#     all_params = {
#         "objective": "rmse",
#         "metric": "rmse",
#         'random_state': RANDOM_STATE,
#         "verbosity": -1,
#         "n_jobs": n_jobs,
#         **params
#     }

#     model = lgb.LGBMRegressor(**all_params)

#     if data_augmentation:
#         train_x, train_y = data_augmentation(train_x, train_y, pct=0.75, scale=0.05)
    
#     model.fit(train_x, train_y)
    
#     preds = model.predict(val_x)
    
#     rmse = mean_squared_error(val_y, preds, squared=False)
    
#     return rmse

In [45]:
# def lgb_calc_OOF(x_train, y_train, x_valid, y_valid, iter, split, RANDOM_STATE=42, noise=False, data_augmentation=False):
#     # split into two validations
#     x_valid1, x_valid2, y_valid1, y_valid2 = train_test_split(x_valid, y_valid, test_size=0.5, random_state=RANDOM_STATE, shuffle=False)

#     '''Use v1, test on v2'''

#     # Bayesian Opt using v1
#     study = optuna.create_study(direction='minimize')
#     study.optimize(lambda trial: objective(trial, x_train, y_train, x_valid1, y_valid1,RANDOM_STATE,data_augmentation), n_trials=20)
#     best_trial1 = study.best_trial.params

#     # Get best params
#     best_params = {
#         "objective": "rmse",
#         "metric": "rmse",
#         'random_state': RANDOM_STATE,
#         "verbosity": -1,
#         "n_jobs": n_jobs,
#         **best_trial1
#     }
        
#     model = lgb.LGBMRegressor(**best_params)
#     if data_augmentation:
#         x_train, y_train = data_augmentation(x_train, y_train, pct=0.75, scale=0.05)
#     if noise:
#         y_train = y_train + np.random.normal(0, 0.25, y_train.shape)
#     model.fit(x_train, y_train)
#     model.booster_.save_model(f'./{dir_to_save}/lgb_iter{iter}_split{split}_val1.txt')

#     # Predict on v2
#     valid2_predict = model.predict(x_valid2)


#     '''Use v2, test on v1'''

#     # Bayesian Opt using v2
#     study = optuna.create_study(direction='minimize')
#     study.optimize(lambda trial: objective(trial, x_train, y_train, x_valid2, y_valid2,RANDOM_STATE,data_augmentation), n_trials=20)
#     best_trial2 = study.best_trial.params

#     # Get best params
#     best_params = {
#         "objective": "rmse",
#         "metric": "rmse",
#         'random_state': RANDOM_STATE,
#         "verbosity": -1,
#         "n_jobs": n_jobs,
#         **best_trial2
#     }
        
#     model = lgb.LGBMRegressor(**best_params)
#     if data_augmentation:
#         x_train, y_train = data_augmentation(x_train, y_train, pct=0.75, scale=0.05)
#     if noise:
#         y_train = y_train + np.random.normal(0, 0.25, y_train.shape)
#     model.fit(x_train, y_train)
#     model.booster_.save_model(f'./{dir_to_save}/lgb_iter{iter}_split{split}_val2.txt')

#     # Predict on v1
#     valid1_predict = model.predict(x_valid1)

#     # Return OOFs
#     return np.concatenate([valid1_predict,valid2_predict])

In [46]:
score_thresh = 2

In [47]:
df_high = train_feats[train_feats['score']>score_thresh].copy()
df_low = train_feats[train_feats['score']<=score_thresh].copy()

In [48]:
def lgb_calc_OOF(x_train, y_train, x_valid, y_valid, x_high, model, early_stop=10, RANDOM_STATE=42, iter=0, split=0):
    # split into two validations
    x_valid1, x_valid2, y_valid1, y_valid2 = model_selection.train_test_split(x_valid, y_valid, test_size=0.5, random_state=RANDOM_STATE, shuffle=False)

    early_stopping_callback = lgb.early_stopping(early_stop, first_metric_only=True, verbose=False)

     # Use v1, test on v2
    model.fit(x_train, y_train, eval_set=[(x_valid1, y_valid1)],
                callbacks=[early_stopping_callback])
    
    model.booster_.save_model(f'./{dir_to_save}/lgb_iter{iter}_split{split}_val2.txt')

    valid2_predict = model.predict(x_valid2)
    high_predict = model.predict(x_high)

     # Use v2, test on v1
    model.fit(x_train, y_train, eval_set=[(x_valid2, y_valid2)],
                callbacks=[early_stopping_callback])
    
    model.booster_.save_model(f'./{dir_to_save}/lgb_iter{iter}_split{split}_val1.txt')

    valid1_predict = model.predict(x_valid1)
    high_predict += model.predict(x_high)

    # Return OOFs
    valid_pred = np.concatenate([valid1_predict,valid2_predict])
    high_predict = high_predict/2
    
    return valid_pred, high_predict

In [49]:
OOF_HIGH_SCORE = np.zeros(len(df_high))
OOF_LOW_SCORE = np.zeros(len(df_low))

models_dict = {}
preds_dict = {}
count = 0
for _ in df_low['id'].unique():
    preds_dict[count] = []
    count+=1


for i in range(iterations):

    kf = model_selection.KFold(n_splits=n_splits, random_state=41 + i, shuffle=True)
    j = 0
    for fold, (train_idx, valid_idx) in enumerate(kf.split(df_low)):
        
        X_train, y_train = df_low.iloc[train_idx][train_cols], df_low.iloc[train_idx][target_col]
        if noise:
            y_train = y_train + np.random.normal(0, 0.25)
        X_valid, y_valid = df_low.iloc[valid_idx][train_cols], df_low.iloc[valid_idx][target_col]
        best_params = {
                "objective": "rmse",
                "metric": "rmse",
                'random_state': 41 + i,
                "n_estimators" : 1000,
                "verbosity": -1,
                "n_jobs": -1,
                "subsample": 0.8,
                'colsample_bytree': 0.8, 
                'colsample_bynode': 0.8,
            }
        
        model = lgb.LGBMRegressor(**best_params)
        
        valid_predict, high_pred = lgb_calc_OOF(X_train, y_train, X_valid, y_valid, 
                                                           df_high[train_cols],
                                                           model, early_stop=15, RANDOM_STATE=41 + i,
                                                           iter=i, split=j)

        for count, idx in enumerate(valid_idx):
            OOF_LOW_SCORE[idx] += valid_predict[count] / iterations

        OOF_HIGH_SCORE += high_pred / iterations / n_splits
        
        models_dict[f'{fold}_{i}'] = model
        j+=1

In [50]:
print('OOF of true score <={:.1f} = {:.5f}'.format(score_thresh, metrics.mean_squared_error(df_low[target_col], 
                                                                   OOF_LOW_SCORE,
                                                                   squared=False)))

OOF of true score <=2.0 = 0.40855


In [51]:
df_low['low reg'] = OOF_LOW_SCORE
df_low['flag'] = 1

df_high = train_feats[train_feats['score']>score_thresh].copy()
df_high['low reg'] = OOF_HIGH_SCORE
df_high['flag'] = 0

df_all = pd.concat([df_high, df_low], axis=0)

In [52]:
# print('OOF before replacement = {:.5f}'.format(metrics.mean_squared_error(df_all[target_col], 
#                                                                    df_all['pred'],
#                                                                    squared=False)))

In [53]:
# df_all.loc[df_all['pred']<=1.5, 'pred'] = df_all['low reg']

In [54]:
# print('OOF after replacement = {:.5f}'.format(metrics.mean_squared_error(df_all[target_col], 
#                                                                    df_all['pred'], 
#                                                                    squared=False)))

In [55]:
# OOF_SCORE = np.zeros(len(train_feats))

# models_dict = {}
# preds_dict = {}
# count = 0
# for _ in train_feats['id'].unique():
#     preds_dict[count] = []
#     count+=1


# for i in range(iterations):

#     if stratification:
#         kf = model_selection.StratifiedKFold(n_splits=n_splits, random_state=41 + i, shuffle=True)
#         j = 0
#         for fold, (train_idx, valid_idx) in enumerate(kf.split(train_feats, score_transformed)):
            
#             X_train, y_train = train_feats.iloc[train_idx][train_cols], train_feats.iloc[train_idx][target_col]
#             X_valid, y_valid = train_feats.iloc[valid_idx][train_cols], train_feats.iloc[valid_idx][target_col]

#             valid_predict = lgb_calc_OOF(X_train, y_train, X_valid, y_valid, i, j, RANDOM_STATE=41 + i, noise=noise, data_augmentation=data_augmentation)

#             OOF_SCORE[valid_idx] += valid_predict / iterations
#             j+=1

#     else:
#         kf = model_selection.KFold(n_splits=n_splits, random_state=41 + i, shuffle=True)
#         j = 0
#         for fold, (train_idx, valid_idx) in enumerate(kf.split(train_feats)):
            
#             X_train, y_train = train_feats.iloc[train_idx][train_cols], train_feats.iloc[train_idx][target_col]
#             X_valid, y_valid = train_feats.iloc[valid_idx][train_cols], train_feats.iloc[valid_idx][target_col]

#             valid_predict = lgb_calc_OOF(X_train, y_train, X_valid, y_valid, i, j, RANDOM_STATE=41 + i, noise=noise, data_augmentation=data_augmentation)

#             OOF_SCORE[valid_idx] += valid_predict / iterations
#             j+=1

In [56]:
# print('OOF metric LGBM = {:.5f}'.format(metrics.mean_squared_error(train_feats[target_col], 
#                                                                    OOF_SCORE,
#                                                                    squared=False)))

In [57]:
# import pickle

# with open(f'{dir_to_save}/OOF_base_lgb.pkl', 'wb') as f:
#     pickle.dump(OOF_SCORE, f)

In [58]:
# f, ax = plt.subplots(figsize=(6, 6))
# ax.scatter(OOF_SCORE, train_feats['score'])
# add_identity(ax, color='r', ls='--')
# # ax.scatter(stds, train_scores_df['score']-means)
# plt.ylabel('True Values')
# plt.xlabel('Predictions')

# plt.show()